<a href="https://colab.research.google.com/github/andimsteph/diss/blob/main/Max_min_test_two_trend_MC_Test_Control_Draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Conducting Monte Carlo Simulations to Generate and Analyze Single-Case Graphs

#Import packages
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

#This function creates a time series with n points, an autocorrelation of a,
#and a constant of ct

def create_time_series(n, a, ct):

    #Create an empty vector to hold values
    time_series = np.empty((0,))

    #Compute first point (no autocorrelation possible)
    point1 = np.random.normal(size = 1)

    #Add point1 to time series
    time_series = np.hstack((time_series, point1))

    #Repeat the process below for all subsequent points
    for i in range(1, n):

        #Compute autocorrelated point error term
        point = a*time_series[i-1]+np.random.normal(size = 1)

        #Add autcorrelated point to time series
        time_series = np.hstack((time_series, point))

    #Add constant to all points
    time_series = time_series + ct

    #Return the time series
    return(time_series)



In [24]:
#This function creates data for an alternating treatment graph with an
#autocorrelation of a, a trend of tr (in degrees), a constant of ct, a minimum
#of nb_points in each condition, a standardized mean difference of smd,
#and level of r in the control condition. Alternation is semi-random

def create_ME_data(a, trA, trB, ct, nb_points, smd, r, alternation):

  if alternation == 'semi-random':

        #Empty vector for labels
        labels = np.empty((0,))

        #Repeat process for each pair of points
        for i in range(nb_points):

            #Randomly select the order of two conditions
            new_var = 2
            conditions = np.random.choice(['Control', 'Test'], 2, replace = False)

            #Add conditions to labels
            labels = np.hstack((labels, conditions))

        #Run code until minimum number of points is reached for both phases
        while (np.sum(labels == 'Control') < nb_points) | (np.sum(labels == 'Test') < nb_points):

            #Randomly select one condition
            condition = np.random.choice(['Control', 'Test'], 1)

            #Add condition to labels
            labels = np.hstack((labels, condition))

  #Create times series
  time_series = create_time_series(len(labels), a, ct)
  all_values = np.copy(time_series)

  #Indices for Phase A (Control)
  idxA = np.where(labels == 'Control')[0]

  #Indices for Phase B
  idxB = np.where(labels == 'Test')[0]

  #Add smd to values of Phase B
  all_values[idxB] += (smd)

  #Identify middle point around which to pivot trend
  middle_point = np.median(range(len(all_values)))

  #Apply trend to all points separately
  #Phase A
  for j in range(len(idxA)):
    distance = idxA[j] - middle_point
    all_values[idxA[j]] += distance * math.tan(trA * math.pi / 180)

  #Phase B
  for j in range(len(idxB)):
        distance = idxB[j] - middle_point
        all_values[idxB[j]] += distance * math.tan(trB * math.pi / 180)

  #Set max and min values, and normalize
  min_value = np.min(all_values)
  max_value = np.max(all_values)
  all_values = (all_values - min_value) / (max_value - min_value) * 10

  #Recalculate smd
  mean_control = np.mean(all_values[idxA])
  mean_test = np.mean(all_values[idxB])
  current_smd = mean_test - mean_control

  #Adjust values to maintain smd
  if current_smd != smd:
      adjustment = smd - current_smd
      all_values[idxB] += adjustment / 2
      all_values[idxA] -= adjustment / 2

  #Set max and min values, and normalize (to ensure positive values)
  min_value = np.min(all_values)
  max_value = np.max(all_values)
  all_values = (all_values - min_value) / (max_value - min_value) * 10

  #add r after normalization to increase level
  all_values[idxA] += r
  all_values[idxB] += r

  #Combine labels and values in same list
  ME_data = [labels, all_values]

  #Return ME data
  return(ME_data)



In [25]:
#Function to produce ME graph

def ATgraph(ME_data, trA, trB):

    #Identify indices for Control and Test
    Control, = np.where(ME_data[0] == 'Control')
    Test, = np.where(ME_data[0] == 'Test')

    #Extract values for Control and Test
    valuesControl = ME_data[1][Control]
    valuesTest = ME_data[1][Test]

    #Initialize figure
    fig = plt.figure()
    ax = fig.add_subplot(111)

    #Plot data
    plt.plot(Control+1, valuesControl, 'ks-', label = 'Control')
    plt.plot(Test+1, valuesTest, 'ko-', label = 'Test')

    #Add axes titles
    plt.xlabel('Session')
    plt.ylabel('Behavior')

    #Add legend to graph
    ax.legend(loc='lower right', frameon=False)

    #Adjust height of graph
    plt.ylim(0, np.max(ME_data[1]*1.5))

    #Remove right and top borders
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)


In [28]:
# Define ranges for each parameter
nb_points_list = [6, 10]
a_list = [0, 0.2]
tr_listA = [0, 0, 0] #determine number of 0s, angle of trend from effect size data
tr_listB = [-30, 0, 30]
smd_list = [0, 3, 6]
r_list = [0, 2, 4]
ct = 10


# Function to create data and graph
def perform_simulations():
  #Empty list to contain all the input data
  all_ME_data = []

  #Empty vector to contain the true values
  true_values = np.empty((0,))

  #Empty vector to contain trend values for each data series
  trend_values = np.empty((0, 2))

  #pp
  pp = PdfPages('MEgraphTest.pdf')

  # Loop over each combination of parameters
  for nb in nb_points_list:
       for a in a_list:
          for trA in tr_listA:
              for trB in tr_listB:
                  for smd in smd_list:
                      for r in r_list:


                          # Generate ME data for the set of parameters
                          ME_data = create_ME_data(a=a, trA=trA, trB=trB, ct=ct, nb_points=nb, smd=smd, r=r, alternation='semi-random')

                          # Plot graph
                          plt.figure()
                          ATgraph(ME_data, trA, trB)

                          # Title to distinguish different graphs
                          plt.title(f'nb_points={nb}, a={a}, trA={trA}, trB={trB}, smd={smd}, r={r}')

                          #plt.show()
                          pp.savefig()
                          plt.close()

                          #Add data series to list
                          all_ME_data.append(ME_data)

                          #Add true value to vector
                          true_values = np.hstack((true_values, 1 if smd > 0 else 0))

                          #Add trend value to vector
                          trend_values = np.vstack((trend_values, np.array([[trA, trB]])))

  pp.close()

  return all_ME_data, true_values, trend_values

#Call
all_ME_data, true_values, trend_values = perform_simulations()


<ipython-input-25-8e3a8c74fdea>:14: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure()
<ipython-input-28-84e4d65f983e>:38: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [ ]:
from google.colab import files
files.download('MEgraphTest.pdf')

In [ ]:
# TEST - Setting parameters
test_params = {
    'a': 0.2,
    'trA': 0,
    'trB': 30,
    'ct': 10,
    'nb_points': 6,
    'smd': 0,
    'r': -8,
    'alternation': 'semi-random'
}

# TEST - Generate ME data with parameters
ME_data = create_ME_data(
    test_params['a'],
    test_params['trA'],
    test_params['trB'],
    test_params['ct'],
    test_params['nb_points'],
    test_params['smd'],
    test_params['r'],
    test_params['alternation']
)

# TEST - Graph plotting
plt.figure
ATgraph(ME_data, trA, trB)
plt.title(f"Test Graph: {test_params}")
plt.show()

NameError: name 'trA' is not defined